# Step 1: Prepare Data in PySpark

In [1]:
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("EmployeeViews").getOrCreate()

data = [
    Row(EmpID=101, Name="Ravi", Department="Engineering", Project="AI Engine", Salary=95000, HoursPerWeek=42),
    Row(EmpID=102, Name="Sneha", Department="Engineering", Project="Data Platform", Salary=87000, HoursPerWeek=45),
    Row(EmpID=103, Name="Kabir", Department="Marketing", Project="Product Launch", Salary=65000, HoursPerWeek=40),
    Row(EmpID=104, Name="Anita", Department="Sales", Project="Client Outreach", Salary=70000, HoursPerWeek=38),
    Row(EmpID=105, Name="Divya", Department="Engineering", Project="AI Engine", Salary=99000, HoursPerWeek=48),
    Row(EmpID=106, Name="Amit", Department="Marketing", Project="Social Media", Salary=62000, HoursPerWeek=35),
    Row(EmpID=107, Name="Priya", Department="HR", Project="Policy Revamp", Salary=58000, HoursPerWeek=37),
    Row(EmpID=108, Name="Manav", Department="Sales", Project="Lead Gen", Salary=73000, HoursPerWeek=41),
    Row(EmpID=109, Name="Neha", Department="Engineering", Project="Security Suite", Salary=91000, HoursPerWeek=46),
    Row(EmpID=110, Name="Farah", Department="HR", Project="Onboarding", Salary=60000, HoursPerWeek=36)
]

df = spark.createDataFrame(data)
df.show(truncate=False)


+-----+-----+-----------+---------------+------+------------+
|EmpID|Name |Department |Project        |Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|101  |Ravi |Engineering|AI Engine      |95000 |42          |
|102  |Sneha|Engineering|Data Platform  |87000 |45          |
|103  |Kabir|Marketing  |Product Launch |65000 |40          |
|104  |Anita|Sales      |Client Outreach|70000 |38          |
|105  |Divya|Engineering|AI Engine      |99000 |48          |
|106  |Amit |Marketing  |Social Media   |62000 |35          |
|107  |Priya|HR         |Policy Revamp  |58000 |37          |
|108  |Manav|Sales      |Lead Gen       |73000 |41          |
|109  |Neha |Engineering|Security Suite |91000 |46          |
|110  |Farah|HR         |Onboarding     |60000 |36          |
+-----+-----+-----------+---------------+------+------------+



# Step 2: Create Views



In [2]:
# Local temp View
df.createOrReplaceTempView("employees_local")

# Global temp View
df.createOrReplaceGlobalTempView("employees_global")


# Part A: Exercises on Local View ( employees_local )

1. List all employees working on the "AI Engine" project.

In [3]:
spark.sql("SELECT * FROM employees_local WHERE Project = 'AI Engine' ").show()

+-----+-----+-----------+---------+------+------------+
|EmpID| Name| Department|  Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------+------+------------+
|  101| Ravi|Engineering|AI Engine| 95000|          42|
|  105|Divya|Engineering|AI Engine| 99000|          48|
+-----+-----+-----------+---------+------+------------+



2. Show all employees from the "Marketing" department with salaries greater than
60,000.

In [5]:
spark.sql("SELECT * FROM employees_local where Department = 'Marketing' AND salary> 60000").show()

+-----+-----+----------+--------------+------+------------+
|EmpID| Name|Department|       Project|Salary|HoursPerWeek|
+-----+-----+----------+--------------+------+------------+
|  103|Kabir| Marketing|Product Launch| 65000|          40|
|  106| Amit| Marketing|  Social Media| 62000|          35|
+-----+-----+----------+--------------+------+------------+



3. Calculate the average salary for each department.

In [6]:
spark.sql(" select Department, Avg(salary) as AvgSalary from employees_local group by Department").show()

+-----------+---------+
| Department|AvgSalary|
+-----------+---------+
|      Sales|  71500.0|
|Engineering|  93000.0|
|  Marketing|  63500.0|
|         HR|  59000.0|
+-----------+---------+



4. List the top 3 highest paid employees overall.

In [8]:
spark.sql("select * from employees_local order by Salary DESC LIMIT 3").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



5. Find employees who work more than 40 hours per week.


In [9]:
spark.sql("select * from employees_local where HoursPerWeek > 40").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



6. Group by project and display the number of employees per project.

In [13]:
spark.sql("select Project, Count(*) as Employees_count from employees_local group by project  ").show()

+---------------+---------------+
|        Project|Employees_count|
+---------------+---------------+
|  Data Platform|              1|
|      AI Engine|              2|
| Product Launch|              1|
|Client Outreach|              1|
| Security Suite|              1|
|  Policy Revamp|              1|
|       Lead Gen|              1|
|   Social Media|              1|
|     Onboarding|              1|
+---------------+---------------+



7. Drop the local view. Try querying again — what happens?

In [17]:
spark.catalog.dropTempView("employees_local")

spark.sql("SELECT * FROM employees_local").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `employees_local` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [employees_local], [], false


# Part B: Exercises on Global View ( employees_global )

1. Retrieve all "HR" employees working fewer than 38 hours/week.

In [20]:
spark.sql("SELECT * FROM global_temp.employees_global WHERE Department = 'HR' AND HoursPerWeek < 38").show()

+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+



2. Calculate the total salary payout for each department.

In [21]:
spark.sql("select Department, SUM(Salary) as TotalPayout from global_temp.employees_global group by Department").show()

+-----------+-----------+
| Department|TotalPayout|
+-----------+-----------+
|      Sales|     143000|
|Engineering|     372000|
|  Marketing|     127000|
|         HR|     118000|
+-----------+-----------+



3. For each employee, add a derived column Status :
If HoursPerWeek > 45 → 'Overworked'
Otherwise → 'Normal'

In [22]:
spark.sql("""
SELECT *,
       CASE WHEN HoursPerWeek > 45 THEN 'Overworked' ELSE 'Normal' END AS Status
FROM global_temp.employees_global
""").show()

+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|    Status|
+-----+-----+-----------+---------------+------+------------+----------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|    Normal|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|    Normal|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|    Normal|
|  104|Anita|      Sales|Client Outreach| 70000|          38|    Normal|
|  105|Divya|Engineering|      AI Engine| 99000|          48|Overworked|
|  106| Amit|  Marketing|   Social Media| 62000|          35|    Normal|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|    Normal|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|    Normal|
|  109| Neha|Engineering| Security Suite| 91000|          46|Overworked|
|  110|Farah|         HR|     Onboarding| 60000|          36|    Normal|
+-----+-----+-----------+---------------+------+---

4. Count the total number of employees working on each "Project" .

In [23]:
spark.sql("select Project, Count(*) as Total from global_temp.employees_global group by Project").show()

+---------------+-----+
|        Project|Total|
+---------------+-----+
|  Data Platform|    1|
|      AI Engine|    2|
| Product Launch|    1|
|Client Outreach|    1|
| Security Suite|    1|
|  Policy Revamp|    1|
|       Lead Gen|    1|
|   Social Media|    1|
|     Onboarding|    1|
+---------------+-----+



5. List employees whose salary is above the average salary in their department.

In [27]:
spark.sql("""
WITH dept_avg AS (
  select Department, AVG(Salary) as avg_salary
  from global_temp.employees_global
  group by Department
)
select e.*
from global_temp.employees_global e
join dept_avg a on e.Department = a.Department
where e.Salary> a.avg_salary
""").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  103|Kabir|  Marketing|Product Launch| 65000|          40|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  110|Farah|         HR|    Onboarding| 60000|          36|
+-----+-----+-----------+--------------+------+------------+



6. Open a new Spark session and query "global_temp.employees_global" from there.

In [36]:
new_spark = SparkSession.builder.appName("NewSession").getOrCreate()

new_spark.sql("SELECT * FROM global_temp.employees_global").show()


+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  106| Amit|  Marketing|   Social Media| 62000|          35|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
|  110|Farah|         HR|     Onboarding| 60000|          36|
+-----+-----+-----------+---------------+------+------------+



# Bonus Challenges

In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

1. Use a window function to assign rank to employees within each department based
on salary.

In [33]:
from pyspark.sql.functions import col

window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())
df.withColumn("SalaryRank", rank().over(window_spec)).select("Name", "Department", "Salary", "SalaryRank").show()

+-----+-----------+------+----------+
| Name| Department|Salary|SalaryRank|
+-----+-----------+------+----------+
|Divya|Engineering| 99000|         1|
| Ravi|Engineering| 95000|         2|
| Neha|Engineering| 91000|         3|
|Sneha|Engineering| 87000|         4|
|Farah|         HR| 60000|         1|
|Priya|         HR| 58000|         2|
|Kabir|  Marketing| 65000|         1|
| Amit|  Marketing| 62000|         2|
|Manav|      Sales| 73000|         1|
|Anita|      Sales| 70000|         2|
+-----+-----------+------+----------+



2. Create another view (local or global) that only contains "Engineering"
employees.

In [34]:
df.filter(col("Department") == "Engineering").createOrReplaceTempView("engineering_employees")

3. Create a SQL view that filters out all employees working < 38 hours and saves
it as "active_employees" .

In [35]:
df.filter(col("HoursPerWeek") >= 38).createOrReplaceTempView("active_Employees")
spark.sql("select * from active_employees").show()

+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
+-----+-----+-----------+---------------+------+------------+

